In [1]:
from datasets import load_dataset, Audio
from transformers import MimiModel, AutoFeatureExtractor
librispeech_dummy = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

# load model and feature extractor
model = MimiModel.from_pretrained("kyutai/mimi")
feature_extractor = AutoFeatureExtractor.from_pretrained("kyutai/mimi")

/home/apurva/miniconda3_11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/apurva/miniconda3_11/lib/python3.11/site-packages/transformers/models/mimi/modeling_mimi.py:217: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [2]:
_ = model.to('cuda:0')

In [3]:
import glob
import torchaudio
from tqdm import tqdm
import torch
import numpy as np

In [4]:
audio_files = glob.glob('/home/apurva/.cache/indri/shrutilipi/audio/*.wav')

In [5]:
len(audio_files)

260806

In [6]:
def convert_audio(wav: torch.Tensor, sr: int, target_sr: int, target_channels: int):
    assert wav.shape[0] in [1, 2], "Audio must be mono or stereo."
    if target_channels == 1:
        wav = wav.mean(0, keepdim=True)
    elif target_channels == 2:
        *shape, _, length = wav.shape
        wav = wav.expand(*shape, target_channels, length)
    elif wav.shape[0] == 1:
        wav = wav.expand(target_channels, -1)
    wav = torchaudio.transforms.Resample(sr, target_sr)(wav)
    return wav

In [8]:
with torch.inference_mode():
    for f in tqdm(audio_files):
        x, sr = torchaudio.load(f)
        x = convert_audio(x, sr=sr, target_sr=24000, target_channels=1)[0]
        inputs = feature_extractor(raw_audio=x, sampling_rate=feature_extractor.sampling_rate, return_tensors="pt").to('cuda:0')
        encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"], num_quantizers=8)
        out_f = f.replace('/audio/', '/tokens/mimi/').replace('.wav', '.npy')
        audio_codes = encoder_outputs.audio_codes.cpu().numpy()[0]
        np.save(out_f, audio_codes)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 260806/260806 [1:59:00<00:00, 36.52it/s]
